# LLM NER and classification

Using an LLM to identify the entities of interest.

## Imports and evironment setups

In [ ]:
!pip install pdfplumber -q
!pip install together -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 893.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.3/91.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 7.3 MB/s eta 0:00:00


In [ ]:
import os
import pdfplumber
import numpy as np
import pandas as pd
import json
import re
import asyncio
import together
from together import AsyncTogether, Together
import time
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
from together import Together
async_client = AsyncTogether()

## Costants

In [ ]:
DATA_PATH='/content/drive/MyDrive/Luxdada/SanRaffaele/Data/Dataset/2019/lettere_dimissioni_2019'
OUTPUT_PATH='/content/drive/MyDrive/Luxdada/SanRaffaele/Data/Dataset LLM'

In [ ]:
DATA_PATH='/content/drive/MyDrive/SanRaffaele/Data/Dataset/2019/lettere_dimissioni_2019'
OUTPUT_PATH='/content/drive/MyDrive/SanRaffaele/Data/Dataset LLM'

In [ ]:
promp_base='''
Sei un medico specializzato in cardiochirurgia. Il tuo compito è estrarre **esclusivamente** le seguenti entità dalla **lettera di dimissione** riportata qui sotto.

###**Entità da estrarre (solo queste):**

### Mappa delle entità e tipi

| Entità                                      | Tipo              | Descrizione                                                                                                                                     |
|--------------------------------------------|-------------------|-------------------------------------------------------------------------------------------------------------------------------------------------|
| n_cartella                                 | Number            | Numero identificativo univoco assegnato alla cartella clinica del paziente.                                                                    |
| data_ingresso_cch                          | Date              | Data in cui il paziente è stato ricoverato presso il reparto di Cardiochirurgia.                                                              |
| data_dimissione_cch                        | Date              | Data in cui il paziente è stato dimesso dal reparto di Cardiochirurgia.                                                                        |
| nome                                       | Text              | Nome proprio del paziente.                                                                                                                      |
| cognome                                    | Text              | Cognome del paziente.                                                                                                                           |
| sesso                                      | Categorical_MF    | Sesso biologico del paziente (M = Maschio, F = Femmina).                                                                                        |
| numero di telefono                         | Text              | Recapito telefonico del paziente o di un contatto di riferimento.                                                                              |
| età al momento dell'intervento             | Number            | Età del paziente calcolata alla data dell’intervento chirurgico.                                                                               |
| data_di_nascita                            | Date              | Data di nascita del paziente.                                                                                                                   |
| Diagnosi                                   | Text              | Diagnosi principale alla base dell'indicazione chirurgica.                                                                                      |
| Anamnesi                                   | Text              | Anamnesi patologica remota e prossima, utile per la valutazione del rischio operatorio.                                                        |
| Motivo ricovero                            | Text              | Indicazione clinica per il ricovero in Cardiochirurgia.                                                                                         |
| classe_nyha                                | Categorical_1234  | Classe funzionale NYHA per scompenso cardiaco (I-IV), definisce la gravità dei sintomi.                                                        |
| angor                                      | Boolean           | Presenza di angina pectoris (dolore toracico di origine ischemica).                                                                            |
| STEMI/NSTEMI                               | Boolean           | Presenza di infarto miocardico acuto con/senza sopraslivellamento del tratto ST.                                                               |
| scompenso_cardiaco_nei_3_mesi_precedenti   | Boolean           | Episodi di scompenso cardiaco documentati nei 3 mesi precedenti l’intervento.                                                                  |
| fumo                                       | Categorical_012   | Abitudine al fumo (0 = mai fumato, 1 = ex-fumatore, 2 = fumatore attivo).                                                                      |
| diabete                                    | Boolean           | Presenza di diabete mellito noto.                                                                                                               |
| ipertensione                               | Boolean           | Presenza di ipertensione arteriosa.                                                                                                             |
| dislipidemia                               | Boolean           | Presenza di dislipidemia (colesterolo e/o trigliceridi elevati).                                                                               |
| BPCO                                       | Boolean           | Presenza di broncopneumopatia cronica ostruttiva.                                                                                               |
| stroke_pregresso                           | Boolean           | Precedente episodio di ictus cerebrale ischemico o emorragico.                                                                                  |
| TIA_pregresso                              | Boolean           | Episodio pregresso di attacco ischemico transitorio (TIA).                                                                                      |
| vasculopatiaperif                          | Boolean           | Malattia vascolare periferica documentata (es. arteriopatia arti inferiori).                                                                   |
| neoplasia_pregressa                        | Boolean           | Presenza di neoplasie trattate in passato.                                                                                                      |
| irradiazionetoracica                       | Boolean           | Pregressa radioterapia al torace, rilevante per effetti tardivi su cuore e vasi.                                                               |
| insufficienza_renale_cronica               | Boolean           | Presenza di insufficienza renale cronica diagnosticata.                                                                                         |
| familiarita_cardiovascolare                | Boolean           | Familiarità per malattie cardiovascolari premature (prima dei 55 anni per uomini, 65 per donne).                                                |
| limitazione_mobilita                       | Boolean           | Presenza di limitazioni significative alla mobilità (es. pazienti allettati).                                                                  |
| endocardite                                | Boolean           | Pregressa o attiva endocardite infettiva, rilevante per indicazione chirurgica.                                                                |
| ritmo_all_ingresso                         | Categorical_012   | Ritmo cardiaco al momento del ricovero (0 = ritmo sinusale, 1 = FA, 2 = altro).                                                                 |
| fibrillazione_atriale                      | Categorical_012   | Presenza di fibrillazione atriale (0 = mai, 1 = parossistica, 2 = permanente/persistente).                                                     |
| dialisi                                    | Boolean           | Paziente in trattamento emodialitico o peritoneale.                                                                                             |
| elettivo_urgenza_emergenza                 | Categorical_012   | Tipo di intervento (0 = elettivo, 1 = urgente, 2 = emergenza).                                                                                   |
| pm                                         | Boolean           | Presenza di pacemaker.                                                                                                                          |
| crt                                        | Boolean           | Presenza di terapia di resincronizzazione cardiaca (CRT).                                                                                       |
| icd                                        | Boolean           | Presenza di defibrillatore impiantabile (ICD).                                                                                                  |
| pci_pregressa                              | Boolean           | Precedente angioplastica coronarica percutanea (PCI).                                                                                           |
| REDO                                       | Boolean           | Intervento cardiochirurgico di revisione (non prima chirurgia).                                                                                |
| Anno REDO                                  | Date              | Anno in cui è stato eseguito l'intervento REDO precedente.                                                                                      |
| Tipo di REDO                               | Text              | Descrizione del tipo di intervento REDO eseguito.                                                                                               |
| Terapia                                    | Text              | Terapia farmacologica in atto al momento del ricovero.                                                                                          |
| lasix                                      | Boolean           | Uso documentato di furosemide (Lasix).                                                                                                          |
| lasix_dosaggio                             | Number            | Dosaggio giornaliero di furosemide in mg.                                                                                                       |
| nitrati                                    | Boolean           | Assunzione di nitrati (vasodilatatori usati per l'angina).                                                                                      |
| antiaggregante                             | Boolean           | Presenza di terapia antiaggregante (es. ASA, clopidogrel).                                                                                      |
| dapt                                       | Boolean           | Doppia antiaggregazione piastrinica (es. ASA + clopidogrel/prasugrel).                                                                          |
| anticoagorali                              | Boolean           | Terapia anticoagulante in corso (es. warfarin, DOAC).                                                                                           |
| aceinib                                    | Boolean           | Uso di ACE-inibitori.                                                                                                                           |
| betabloc                                   | Boolean           | Uso di beta-bloccanti.                                                                                                                          |
| sartanici                                  | Boolean           | Uso di sartani (ARBs).                                                                                                                          |
| caantag                                    | Boolean           | Uso di calcio-antagonisti.                                                                                                                      |
| esami_all_ingresso                         | Text              | Risultati di laboratorio e strumentali al momento dell’ingresso.                                                                               |
| Decorso_post_operatorio                    | Text              | Descrizione del decorso clinico successivo all’intervento chirurgico.                                                                          |
| IABP/ECMO/IMPELLA                          | Boolean           | Necessità di supporto meccanico circolatorio (IABP, ECMO o Impella).                                                                           |
| Inotropi                                   | Boolean           | Necessità di farmaci inotropi positivi nel post-operatorio.                                                                                     |
| secondo_intervento                         | Boolean           | Esecuzione di un secondo intervento durante la degenza attuale.                                                                                |
| Tipo_secondo_intervento                    | Text              | Tipo e motivazione del secondo intervento chirurgico.                                                                                           |
| II_Run                                     | Boolean           | Presenza di secondo passaggio in circolazione extracorporea (CEC).                                                                             |
| Causa_II_Run_CEC                           | Text              | Motivazione per il secondo utilizzo della CEC.                                                                                                  |
| LCOS                                       | Boolean           | Sindrome da bassa portata cardiaca (Low Cardiac Output Syndrome) post-operatoria.                                                               |
| Impianto_PM_post_intervento                | Boolean           | Necessità di impianto di pacemaker dopo l’intervento.                                                                                           |
| Stroke_TIA_post_op                         | Boolean           | Evento neurologico ischemico (TIA/stroke) avvenuto dopo l’intervento.                                                                          |
| Necessità_di_trasfusioni                   | Boolean           | Necessità di trasfusioni ematiche post-intervento.                                                                                              |
| IRA                                        | Boolean           | Insufficienza renale acuta insorta nel post-operatorio.                                                                                         |
| Insufficienza_respiratoria                 | Boolean           | Insorgenza di insufficienza respiratoria nel post-operatorio.                                                                                   |
| FA_di_nuova_insorgenza                     | Boolean           | Fibrillazione atriale di nuova insorgenza nel post-operatorio.                                                                                  |
| Ritmo_alla_dimissione                      | Categorical_012   | Ritmo cardiaco documentato alla dimissione (0 = sinusale, 1 = FA, 2 = altro).                                                                   |
| H_Stay_giorni (da intervento a dimissione) | Number            | Durata della degenza in giorni, calcolata dall’intervento alla dimissione.                                                                      |
| Morte                                      | Boolean           | Evento di decesso durante la degenza cardiochirurgica.                                                                                          |
| Causa_morte                                | Text              | Causa clinica del decesso (es. sepsi, shock cardiogeno, ecc.).                                                                                  |
| data_morte                                 | Date              | Data del decesso, se avvenuto.                                                                                                                  |
| esami_alla_dimissione                      | Text              | Risultati di laboratorio e strumentali prima della dimissione.                                                                                  |
| terapia_alla_dimissione                    | Text              | Terapia farmacologica prescritta alla dimissione.                                                                                               |



---

### **Istruzioni IMPORTANTI:**

- Ragiona considerando **frase per frase**.
- Non estrarre **nessuna entità** diversa da quelle elencate.
- Se un'entità non è presente nella lettera, **non inventarla** e **non includerla** nel risultato.
- Attenzione però i nomi delle entità che vedi sopra sono in alcuni casi degli acronimi o diminutivi delle entità.
- Il formato di output deve essere una lista JSON, dove ogni elemento è un oggetto con **due chiavi**:
    - `"entità"`: il nome dell'entità
    - `"valore"`: il valore estratto dell'entità
**NON** aggiungere commenti, spiegazioni, note, intestazioni o altro: **solo** la lista JSON.

---

###Esempio di input(esempio parziale della lettera di dimission)
Si dimette in data 02/09/2019
il Sig. BERTOLOTTI FRANCO
Nato il 27/03/1939 telefono 3479927663
ricoverato presso questo ospedale dal 27/08/2019
Numero Cartella 2019034139

Diagnosi alla dimissione:
Intervento di plastica valvolare mitralica per via percutanea mediante posizionamento di duplice dispositivo Mitraclip.

Motivo del Ricovero:
Insufficienza mitralica in status post rivascolarizzazione miocardica chirurgica mediante triplice bypass coronarico.

Cenni Anamnestici:
Paziente nega farmacoallergie.
Familiarità positiva per cardiopatia ischemica (padre).
Ex fumatore, stop nel 1990 (1 pack/die).
Diabete mellito in tp ipoglicemizzante orale.
IRC (crea all'ingresso 2,64 mg/dl).


---

###Esmpio output(esempio parziale in JSON):

```json
[
  { "entità": "data_dimissione_cch", "valore": "02/09/2019" },
  { "entità": "nome", "valore": "FRANCO" },
  { "entità": "cognome", "valore": "BERTOLOTTI" },
  { "entità": "data_di_nascita", "valore": "27/03/1939" },
  { "entità": "numero di telefono", "valore": "3479927663" },
  { "entità": "data_ingresso_cch", "valore": "27/08/2019" },
  { "entità": "n_cartella", "valore": "2019034139" },
  { "entità": "Diagnosi text", "valore": "Intervento di plastica valvolare mitralica per via percutanea mediante posizionamento di duplice dispositivo Mitraclip." },
  { "entità": "Motivo ricovero", "valore": "Insufficienza mitralica in status post rivascolarizzazione miocardica chirurgica mediante triplice bypass coronarico." },
  { "entità": "fumo", "valore": true },
  { "entità": "diabete", "valore": true },
  { "entità": "insufficienza renale cronica", "valore": true },
  { "entità": "familiarita cardiovascolare", "valore": true }
]



'''

## Functions

In [ ]:
def extract_text_pdf(file_path):
    all_text = ''
    with pdfplumber.open(file_path) as pdf:
        for num_page, page in enumerate(pdf.pages):
            page_text = page.extract_text()
            if page_text:
              all_text += f'{{{num_page}}}-----------------------------------\n{page_text}\n\n'
    return all_text

In [ ]:
def extract_text_md(folder_path):
  list_file=os.listdir(folder_path)
  text_list=[]
  for file in list_file:
    path_file=os.path.join(folder_path,file)
    with open(path_file,'r') as f:
      text_list.append(f.read())
  return text_list

In [ ]:
async def get_response(prompt_text:str, model:str)->str:
  for sleep_time in [1, 2, 4]:
    try:
      response = await async_client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt_text}],
            response_format={"type": "json_schema"},
            temperature=0.1,
            top_p=1,
            max_tokens=8192
            )
      break
    except together.error.RateLimitError as e:
        print(e)
        await asyncio.sleep(sleep_time)

  response_text=response.choices[0].message.content
  return response_text

In [ ]:
def create_dataframe(response_text:str, )->pd.DataFrame:
  json_response=json.loads(response_text)
  df = pd.DataFrame(json_response)
  return df

In [ ]:
def save_file(data, path: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, 'wb') as f:
        pickle.dump(data, f)

In [ ]:
file_list=extract_text_md(DATA_PATH)
print(len(file_list))

291


In [ ]:
# Splitta sul pattern tipo {0}----------------------------------- (con newline opzionale dopo)
#Inutile perchè la finestra del contesto è abbastanza grade , il file è circa 4k tokens finestra sopra i 100k nella maggior parte dei casi
pagine = re.split(r'\{\d+\}-+[\n]?', text)
len(pagine)

TypeError: expected string or bytes-like object, got 'list'

In [ ]:
prompt_text=promp_base+text

## DeepSeekV3

In [ ]:
response_list=[]
fine =0
inizio=0

In [ ]:
prompt_list=[promp_base+file for file in file_list]
while fine<len(prompt_list):
  fine=inizio+3
  print(fine)
  if fine>len(prompt_list):
    fine=len(prompt_list)
  for i in range(3):
    try:
      response_list.extend(await asyncio.gather(*[get_response(prompt, 'deepseek-ai/DeepSeek-V3') for prompt in prompt_list[inizio:fine]]))
      break
    except Exception as e:
      print(f"Tentativo {i+1} fallito: {e}")
      time.sleep(2 ** i)
  time.sleep(3)
  inizio=fine

3
6
9
12
15
18
21
24
27
30
33
36
39
Tentativo 1 fallito: Error code: 500 - {"message": "Internal Server Error"}
42
45
48
51
54
57
60
63
66
69
72
75
78
81
84
87
90
93
96
99
102
105
108
111
114
117
120
123
126
129
132
135
138
141
144
147
150
153
156
159
162
165
168
171
174
177
180
183
186
189
192
195
198


ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7ed36bb72c50>


201
204
207
210
213
216
219
222
225
228
231
234
237
240
243
246
249
252
255
258
261
264
Tentativo 1 fallito: Error code: 503 - The server is overloaded or not ready yet.
267
270
273
276
279
282
285
288
291


In [ ]:
len(response_list)

291

In [ ]:
df_response_list=[]
for response in response_list:
  df_response=create_dataframe(response)
  df_response_list.append(df_response)

In [ ]:
df_response_list[0].head()

,entità,valore
0,data_dimissione_cch,02/09/2019
1,nome,FRANCO
2,cognome,BERTOLOTTI
3,data_di_nascita,27/03/1939
4,numero di telefono,3479927663


In [ ]:
save_file(df_response_list, os.path.join(OUTPUT_PATH, 'DeepSeek-V3.pkl'))

##DeepSeek R1

In [ ]:
fine =0
inizio=0
response_list=[]

In [ ]:
prompt_list=[promp_base+file for file in file_list]

while fine<len(prompt_list):
  fine=inizio+2
  print(fine)
  if fine>len(prompt_list):
    fine=len(prompt_list)
  for i in range(3):
    try:
      response_list.extend(await asyncio.gather(*[get_response(prompt, 'deepseek-ai/DeepSeek-R1-0528-tput') for prompt in prompt_list[inizio:fine]]))
      break
    except Exception as e:
      print(f"Tentativo {i+1} fallito: {e}")
      time.sleep(2 ** i)
  time.sleep(2)
  inizio=fine

2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62
64
66
68
70
72
74
76
78
80
82
84
86
88
90
92
94
96
98
100
102
104
106
108
110
112
114
116
118
120
122
124
126
128
130
132
134
136
138
140
142
144
146
148
150
152
154
156
158
160
162
164
166
168
170
172
174
176
178
180
182
184
186
188
190
192
194
196
198
200
202
204
206
208
210
212
214
216
218
220
222
224
226
228
230
232
234
236
238
240
242
244
246
248
250
252
254
256
258
260
262
264
266
268
270
272
274
276
278
280
282
284
286
288
290
292


In [ ]:
df_response_list=[]
for response in response_list:
  df_response=create_dataframe(response)
  df_response_list.append(df_response)

ValueError: If using all scalar values, you must pass an index

In [ ]:
df_response_list[0].head()

In [ ]:
save_file(response_list, os.path.join(OUTPUT_PATH, '_response_list_DeepSeek-R1-0528-tput.pkl'))

In [ ]:
save_file(df_response_list, os.path.join(OUTPUT_PATH, 'DeepSeek-R1-0528-tput.pkl'))

## Llama-3.3-70B-Instruct

In [ ]:
response_list=[]
prompt_list=[promp_base+file for file in file_list]
fine =0
inizio=0
while fine<len(prompt_list):
  fine=inizio+3
  print(fine)
  if fine>len(prompt_list):
    fine=len(prompt_list)
  for i in range(3):
    try:
      response_list.extend(await asyncio.gather(*[get_response(prompt, 'meta-llama/Llama-3.3-70B-Instruct-Turbo') for prompt in prompt_list[inizio:fine]]))
      break
    except Exception as e:
      print(f"Tentativo {i+1} fallito: {e}")
      time.sleep(2 ** i)
  time.sleep(2)
  inizio=fine

3
6
9
12
15
18
21
24
27
30
33
36
39
42
45
48
51
54
57
60
63
66
69
72
75
78
81
84
87
90
93
96
99
102
105
108
111
114
117
120
123
126
129
132
135
138
141
144
147
150
153
156
159
162
165
168
171
174
177
180
183
186
189
192
195
198
201
204
207
210
213
216
219
222
225
228
231
234
237
240
243
246
249
252
255
258
261
Tentativo 1 fallito: Error code: 503 - The server is overloaded or not ready yet.
264
267
270
273
276
279
282
285
288
291


In [ ]:
df_response_list=[]
for response in response_list:
  df_response=create_dataframe(response)
  df_response_list.append(df_response)

In [ ]:
df_response_list[0].head()

,entità,valore
0,data_dimissione_cch,02/09/2019
1,nome,FRANCO
2,cognome,BERTOLOTTI
3,data_di_nascita,27/03/1939
4,numero di telefono,3479927663


In [ ]:
save_file(df_response_list, os.path.join(OUTPUT_PATH, 'LLM_Llama-3.3-70B.pkl'))